In [228]:
import os
from dotenv import load_dotenv
load_dotenv()
import openai
import json
import ast
import time
from tqdm import tqdm

In [225]:
# Set openai.api_key to the OPENAI environment variable
openai.api_key = os.environ["api_key"]

# Extract Education Background from Biography

- system_msg and prompt to instruct LLM to return information in dictionary format

In [226]:
def extract_education_background(text):
    system_msg = "I am an AI assistant designed to help you extract educational background information from a given text. \
    I understand that the text contains a biography and I am capable of identifying and extracting details about the person's education, \
    including their Bachelor's, Master's, and PhD degrees. I also understand that after extracting the following educational information from the text\
    I have to dispaly it in the following format: {'Bachelor Degree': 'Institution', 'Master Degree': 'Institution', 'PhD': 'Institution'}. \
    If there is no information about a particular degree in the text, please return None for the corresponding key."

    prompt = f"""
    Based on the biography below, tell me where did this person got his Bachelor's Degree, Master's Degree and PhD from. Return the results strictly in a dictionary format as such, {{"Bachelor Degree": "Institution", "Master Degree": "Institution", "PhD": "Institution"}}. If there is no information or if information is ambiguous about a particular degree in the text, please return "None" for the corresponding key. Also please make sure key and value is between double quote "string".

    Biography:

    {text}
    """
    response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "system", "content": system_msg },
                    {"role": "user", "content": prompt}
            ],
            temperature=0)
    return response['choices'][0]['message']['content']

In [222]:
text =  "Jie Zhang is a Professor of the School of Computer Science and Engineering at NTU Singapore, leading the Computational Intelligence Group. He obtained Ph.D. in Cheriton School of Computer Science from University of Waterloo and was the recipient of Alumni Gold Medal in 2009. Then he joined NTU as an Assistant Professor and was promoted to Associate Professor in 2015. From 2017-2018, he was appointed as Tan Chin Tuan Exchange Fellowship, New York University. He was also an Adjunct Fellow, Singapore Institute of Manufacturing Technology (SIMTech), A*STAR, from 2020-2021. His papers have been published by top journals and conferences and won several best paper awards. Jie Zhang is also active in serving research communities."

response = extract_education_background(text=text)


In [223]:
print(response)

{"Bachelor Degree": "None", "Master Degree": "None", "PhD": "University of Waterloo"}


In [ ]:
raw_dir = './prof_raw_data/'
prefix = 'education'
gpt_dir = './gpt_data/'

dr_ntu_files = [os.path.join(raw_dir, f) for f in os.listdir(raw_dir) if os.path.isfile(os.path.join(raw_dir, f)) and 'dr_ntu' in f]

for file in tqdm(dr_ntu_files[82:],position=0,leave=True):
    with open(file,'r') as f:
        prof_dict = json.load(f)
    biography = prof_dict['biography']
    name = prof_dict['full_name'].lower().replace(' ','_')
    education_detail_str = extract_education_background(biography)
    education_detail_dict = ast.literal_eval(education_detail_str.strip())
    for key,value in education_detail_dict.items():
        if value =="None":
            education_detail_dict[key] = None
    with open(f"{gpt_dir}{prefix}_{name}.json",'w') as f:
        json.dump(education_detail_dict,f)
    # only limited to 3 calls per minute
    time.sleep(25)
